In [71]:
import pandas as pd
import numpy as np

In [72]:
df = pd.DataFrame({'A': np.random.randint(0, 10, 10)})
df

,A
0,6
1,2
2,0
3,8
4,0
5,7
6,4
7,7
8,2
9,3


In [73]:
df['B'] = df['A'].diff()

In [89]:
df

,A,B,exp_moving_avg,ewm_diff
0,6,NaN,NaN,NaN
1,2,-4.0,NaN,NaN
2,0,-2.0,NaN,NaN
3,8,8.0,NaN,NaN
4,0,-8.0,4.602941,-4.602941
5,7,7.0,2.606523,4.393477
6,4,-3.0,4.449919,-0.449919
7,7,3.0,4.264769,2.735231
8,2,-5.0,5.377552,-3.377552
9,3,1.0,4.012777,-1.012777


In [48]:
x, x_diff = df['A'].iloc[0], df['B'].iloc[1:]

In [49]:
np.r_[x, x_diff]

array([ 8., -8.,  8., -6.,  0.,  7., -6., -3.,  0.,  6.])

In [50]:
x

8

In [51]:
x_diff

1   -8.0
2    8.0
3   -6.0
4    0.0
5    7.0
6   -6.0
7   -3.0
8    0.0
9    6.0
Name: B, dtype: float64

In [52]:
# .r_ concatenates x= first obervation to the beginning of the x_diff column
np.r_[x, x_diff].cumsum().astype(int)

array([8, 0, 8, 2, 2, 9, 3, 0, 0, 6])

In [53]:
df['C'] = np.r_[x, x_diff].cumsum().astype(int)
df

,A,B,C
0,8,NaN,8
1,0,-8.0,0
2,8,8.0,8
3,2,-6.0,2
4,2,0.0,2
5,9,7.0,9
6,3,-6.0,3
7,0,-3.0,0
8,0,0.0,0
9,6,6.0,6


## Rescaling from Moving Average


In [88]:
window_setting = 4
df['exp_moving_avg'] = df['A'].ewm(span=4, min_periods=4, adjust=True).mean().shift()
df['ewm_diff'] = df['A'] - df['exp_moving_avg']
df

,A,B,exp_moving_avg,ewm_diff
0,6,NaN,NaN,NaN
1,2,-4.0,NaN,NaN
2,0,-2.0,NaN,NaN
3,8,8.0,NaN,NaN
4,0,-8.0,4.602941,-4.602941
5,7,7.0,2.606523,4.393477
6,4,-3.0,4.449919,-0.449919
7,7,3.0,4.264769,2.735231
8,2,-5.0,5.377552,-3.377552
9,3,1.0,4.012777,-1.012777


In [74]:
window_setting = 4
train_set = df.iloc[:window_setting]


test_set = test_set['ewm_diff'].reset_index().iloc[:, 1:]



KeyError: 'ewm_diff'

In [90]:
original_tail = train_set['A'].tail(window_setting)

rolling_mean_add = original_tail.ewm(span=4, adjust=True).mean()[3]
print(rolling_mean_add)
unscaled = test_set.iloc[0,:] + rolling_mean_add
unscaled


4.602941176470588


IndexingError: Too many indexers

In [18]:
original_tail = original_tail.append(pd.Series([float(unscaled)]))
original_tail

0    1.0
1    1.0
2    6.0
3    8.0
0    4.0
dtype: float64

In [19]:
# Unscaled values (tail) to calculate moving average from

print(original_tail.tail(5))


0    1.0
1    1.0
2    6.0
3    8.0
0    4.0
dtype: float64


In [22]:

rolling_mean = original_tail.ewm(span=4, min_periods= 4, adjust=True).mean()
rolling_mean = rolling_mean.reset_index().iloc[:,1]

unscaled = rolling_mean[4] + test_set.iloc[1,:]

print(rolling_mean)
print()
print('Exponential Weighted Mean of most recent 4 unscaled values')
print(rolling_mean[4])

0         NaN
1         NaN
2         NaN
3    5.595588
4    4.903539
Name: 0, dtype: float64

Exponential Weighted Mean of most recent 4 unscaled values
4.903539208882719


In [90]:
test_set.iloc[1,:]

ewm_diff    0.355309
Name: 1, dtype: float64

In [25]:
original_tail = original_tail.append(pd.Series([float(unscaled)]))
original_tail

0    1.0
1    1.0
2    6.0
3    8.0
0    4.0
0    8.0
dtype: float64

In [27]:
original_tail.tail(6)

0    1.0
1    1.0
2    6.0
3    8.0
0    4.0
0    8.0
dtype: float64

In [28]:
rolling_mean = original_tail.ewm(span=4, min_periods=4, adjust=True).mean()
rolling_mean = rolling_mean.reset_index().iloc[:,1]

#unscaled = rolling_mean[4] + test_set.iloc[1,:]

print(rolling_mean)
print()
#print('Exponential Weighted Mean of most recent 4 unscaled values')
#print(rolling_mean[5])

0         NaN
1         NaN
2         NaN
3    5.595588
4    4.903539
5    6.202739
Name: 0, dtype: float64



#### Rescaling Back to Scale

In [39]:
test_diff = test_diff.squeeze()

In [40]:

for item in test_diff:
    print(item)

-2.5280000000000005
0.4832000000000001
4.28992
-0.42604799999999976
1.7443711999999998
-4.95337728


In [69]:
train_set = df.iloc[:window_setting]
test_set = test_set['ewm_diff'].reset_index().iloc[:, 1:]

original_tail = train_set['A'].tail(window_setting)

for item in test_set:
    ewm_rolling_avg = original_tail.ewm(span=4, min_periods=4, adjust=True).mean()
    ewm_rolling_avg = ewm_rolling_avg.reset_index().iloc[:,1]
    ewm_rolling_avg = ewm_rolling_avg[len(original_tail)-1]
    
    # original_tail.tail(4).ewm(span=4, adjust=False).mean()[window_setting-1]
    #print(original_tail.tail(4))
    print('rolling avg', ewm_rolling_avg)
    #print(item)
    #print('sum', item + ewm_rolling_avg)
    
    unscaled_result = item + ewm_rolling_avg
    original_tail = original_tail.append(pd.Series([float(unscaled_result)]))

#df['unscaled_y'] = list(original_tail)
#df

rolling avg 3.9191176470588225
rolling avg 3.0867453157529487
rolling avg 5.56780343716434
rolling avg 4.511101161473834
rolling avg 2.675835277453758
rolling avg 1.5946048549684968


In [70]:
original_tail

0    8.0
1    0.0
2    8.0
3    2.0
0    2.0
0    9.0
0    3.0
0    0.0
0    0.0
0    6.0
dtype: float64

### DONE. Now for LME

In [115]:
LME_futures = pd.read_excel('forecasting_raw_data/LME Futures Price.xlsx')
LME_futures = LME_futures.iloc[3:, 1:]
#LME_futures['Unnamed: 1'] = pd.datetime(LME_futures['Unnamed: 1'])
LME_futures.index = LME_futures['Unnamed: 1']



LME_futures = LME_futures.iloc[:, 1:]
LME_futures = LME_futures[LME_futures.index.year>=2000]
LME_futures.isnull().sum()
LME = LME_futures.iloc[:, 0]
LME = LME.astype(float)
LME = LME.resample('B').mean()

def preprocess(series, window_setting):
    moving_avg =series.ewm(span=4, min_periods=4, adjust=True).mean().shift()
    moving_avg_diff = series - moving_avg
    return moving_avg_diff, moving_avg

In [116]:
def preprocess(series, window_setting):
    moving_avg =series.ewm(span=4, min_periods=4, adjust=True).mean().shift()
    moving_avg_diff = series - moving_avg
    return moving_avg_diff, moving_avg

In [117]:
LME_stationary, moving_avg_list = preprocess(LME, 4)

#rcParams['figure.figsize'] = 9, 6

window_setting= 4

In [94]:
LME_stationary.head()

Unnamed: 1
2000-01-03           NaN
2000-01-04           NaN
2000-01-05           NaN
2000-01-06           NaN
2000-01-07   -121.759191
Freq: B, Name: Cash Price ($/MT), dtype: float64

In [120]:
moving_avg_list_test = moving_avg_list['2017-10-31':]
moving_avg_list_train = moving_avg_list[:'2017-10-30']

In [122]:
#train_set = df.iloc[:window_setting]
#test_set = test_set['ewm_diff']


y_test = LME_stationary['2017-10-31':]
y_train = LME_stationary[:'2017-10-30']


#.reset_index().iloc[:, 1:]

# = train_set['A'].tail(window_setting)
original_tail = LME.loc[y_train.index[-window_setting:]]
ewm_list = []

for item in y_test:
    ewm_rolling_avg = original_tail.ewm(span=4, min_periods=4, adjust=True).mean()
    ewm_rolling_avg = ewm_rolling_avg.reset_index().iloc[:,1]
    ewm_rolling_avg = ewm_rolling_avg[len(original_tail)-1]
    
    # original_tail.tail(4).ewm(span=4, adjust=False).mean()[window_setting-1]
    #print(original_tail.tail(4))
    print(original_tail)
    print('rolling avg', ewm_rolling_avg)
    print()
    #print(item)
    #print('sum', item + ewm_rolling_avg)
    ewm_list.append(float(ewm_rolling_avg))
    
    unscaled_result = item + ewm_rolling_avg
    original_tail = original_tail.append(pd.Series([float(unscaled_result)]))

original_tail = pd.Series(original_tail)
original_tail = original_tail[window_setting:]
original_tail.index = y_test.index
original_tail.head()

Unnamed: 1
2017-10-25    11821.0
2017-10-26    11716.0
2017-10-27    11546.0
2017-10-30    11630.5
Freq: B, Name: Cash Price ($/MT), dtype: float64
rolling avg 11640.25551470588

2017-10-25 00:00:00    11821.000000
2017-10-26 00:00:00    11716.000000
2017-10-27 00:00:00    11546.000000
2017-10-30 00:00:00    11630.500000
0                      12236.163781
dtype: float64
rolling avg 11898.716768363382

2017-10-25 00:00:00    11821.000000
2017-10-26 00:00:00    11716.000000
2017-10-27 00:00:00    11546.000000
2017-10-30 00:00:00    11630.500000
0                      12236.163781
0                      12757.761728
dtype: float64
rolling avg 12259.151180227156

2017-10-25 00:00:00    11821.000000
2017-10-26 00:00:00    11716.000000
2017-10-27 00:00:00    11546.000000
2017-10-30 00:00:00    11630.500000
0                      12236.163781
0                      12757.761728
0                      12584.578156
dtype: float64
rolling avg 12393.070864671943

2017-10-25 00:00:00    11821.000

rolling avg 11745.557690159665

2017-10-25 00:00:00    11821.000000
2017-10-26 00:00:00    11716.000000
2017-10-27 00:00:00    11546.000000
2017-10-30 00:00:00    11630.500000
0                      12236.163781
0                      12757.761728
0                      12584.578156
0                      12702.327050
0                      12892.440271
0                      12622.961425
0                      12674.347379
0                      12270.881430
0                      12079.041571
0                      12455.319153
0                      11742.357234
0                      11656.736778
0                      11317.183772
0                      11541.641701
0                      11624.135678
0                      11833.635521
0                      11804.138528
0                      11895.639352
0                      12002.140130
0                      11536.140747
0                      11313.140085
0                      11483.139594
0                      11073.139


2017-10-25 00:00:00    11821.000000
2017-10-26 00:00:00    11716.000000
2017-10-27 00:00:00    11546.000000
2017-10-30 00:00:00    11630.500000
0                      12236.163781
0                      12757.761728
0                      12584.578156
0                      12702.327050
0                      12892.440271
0                      12622.961425
0                      12674.347379
0                      12270.881430
0                      12079.041571
0                      12455.319153
0                      11742.357234
0                      11656.736778
0                      11317.183772
0                      11541.641701
0                      11624.135678
0                      11833.635521
0                      11804.138528
0                      11895.639352
0                      12002.140130
0                      11536.140747
0                      11313.140085
0                      11483.139594
0                      11073.139533
0                      1125

Length: 122, dtype: float64
rolling avg 13645.875239737168

2017-10-25 00:00:00    11821.000000
2017-10-26 00:00:00    11716.000000
2017-10-27 00:00:00    11546.000000
2017-10-30 00:00:00    11630.500000
0                      12236.163781
0                      12757.761728
0                      12584.578156
0                      12702.327050
0                      12892.440271
0                      12622.961425
0                      12674.347379
0                      12270.881430
0                      12079.041571
0                      12455.319153
0                      11742.357234
0                      11656.736778
0                      11317.183772
0                      11541.641701
0                      11624.135678
0                      11833.635521
0                      11804.138528
0                      11895.639352
0                      12002.140130
0                      11536.140747
0                      11313.140085
0                      11483.139594
0   

Length: 150, dtype: float64
rolling avg 14639.955314002189

2017-10-25 00:00:00    11821.000000
2017-10-26 00:00:00    11716.000000
2017-10-27 00:00:00    11546.000000
2017-10-30 00:00:00    11630.500000
0                      12236.163781
0                      12757.761728
0                      12584.578156
0                      12702.327050
0                      12892.440271
0                      12622.961425
0                      12674.347379
0                      12270.881430
0                      12079.041571
0                      12455.319153
0                      11742.357234
0                      11656.736778
0                      11317.183772
0                      11541.641701
0                      11624.135678
0                      11833.635521
0                      11804.138528
0                      11895.639352
0                      12002.140130
0                      11536.140747
0                      11313.140085
0                      11483.139594
0   

...

rolling avg 12038.108285646214

2017-10-25 00:00:00    11821.000000
2017-10-26 00:00:00    11716.000000
2017-10-27 00:00:00    11546.000000
2017-10-30 00:00:00    11630.500000
0                      12236.163781
0                      12757.761728
0                      12584.578156
0                      12702.327050
0                      12892.440271
0                      12622.961425
0                      12674.347379
0                      12270.881430
0                      12079.041571
0                      12455.319153
0                      11742.357234
0                      11656.736778
0                      11317.183772
0                      11541.641701
0                      11624.135678
0                      11833.635521
0                      11804.138528
0                      11895.639352
0                      12002.140130
0                      11536.140747
0                      11313.140085
0                      11483.139594
0                      11073.139

2017-10-25 00:00:00    11821.000000
2017-10-26 00:00:00    11716.000000
2017-10-27 00:00:00    11546.000000
2017-10-30 00:00:00    11630.500000
0                      12236.163781
0                      12757.761728
0                      12584.578156
0                      12702.327050
0                      12892.440271
0                      12622.961425
0                      12674.347379
0                      12270.881430
0                      12079.041571
0                      12455.319153
0                      11742.357234
0                      11656.736778
0                      11317.183772
0                      11541.641701
0                      11624.135678
0                      11833.635521
0                      11804.138528
0                      11895.639352
0                      12002.140130
0                      11536.140747
0                      11313.140085
0                      11483.139594
0                      11073.139533
0                      11252

2017-10-25 00:00:00    11821.000000
2017-10-26 00:00:00    11716.000000
2017-10-27 00:00:00    11546.000000
2017-10-30 00:00:00    11630.500000
0                      12236.163781
0                      12757.761728
0                      12584.578156
0                      12702.327050
0                      12892.440271
0                      12622.961425
0                      12674.347379
0                      12270.881430
0                      12079.041571
0                      12455.319153
0                      11742.357234
0                      11656.736778
0                      11317.183772
0                      11541.641701
0                      11624.135678
0                      11833.635521
0                      11804.138528
0                      11895.639352
0                      12002.140130
0                      11536.140747
0                      11313.140085
0                      11483.139594
0                      11073.139533
0                      11252

2017-10-25 00:00:00    11821.000000
2017-10-26 00:00:00    11716.000000
2017-10-27 00:00:00    11546.000000
2017-10-30 00:00:00    11630.500000
0                      12236.163781
0                      12757.761728
0                      12584.578156
0                      12702.327050
0                      12892.440271
0                      12622.961425
0                      12674.347379
0                      12270.881430
0                      12079.041571
0                      12455.319153
0                      11742.357234
0                      11656.736778
0                      11317.183772
0                      11541.641701
0                      11624.135678
0                      11833.635521
0                      11804.138528
0                      11895.639352
0                      12002.140130
0                      11536.140747
0                      11313.140085
0                      11483.139594
0                      11073.139533
0                      11252

Unnamed: 1
2017-10-31    12236.163781
2017-11-01    12757.761728
2017-11-02    12584.578156
2017-11-03    12702.327050
2017-11-06    12892.440271
Freq: B, dtype: float64

In [112]:
ewm_list = pd.Series(ewm_list)
ewm_list.index = y_test.index

In [100]:
original_tail.head()

Unnamed: 1
2017-10-31    12236.163781
2017-11-01    12757.761728
2017-11-02    12584.578156
2017-11-03    12702.327050
2017-11-06    12892.440271
Freq: B, dtype: float64

In [113]:
ewm_list.head()

Unnamed: 1
2017-10-31    11640.255515
2017-11-01    11898.716768
2017-11-02    12259.151180
2017-11-03    12393.070865
2017-11-06    12518.886559
Freq: B, dtype: float64

In [121]:
moving_avg_list_test.head()

Unnamed: 1
2017-10-31    11660.591734
2017-11-01    11898.955040
2017-11-02    12242.573024
2017-11-03    12372.743814
2017-11-06    12496.446289
Freq: B, Name: Cash Price ($/MT), dtype: float64

In [101]:
LME[LME.index=='2017-10-31']

Unnamed: 1
2017-10-31    12256.5
Freq: B, Name: Cash Price ($/MT), dtype: float64

In [ ]:
X_test = X.loc['2017-10-31':  ,:]
y_test = y['2017-10-31':]

y_train = y[:'2017-10-30']
X_train = X.loc[:'2017-10-30', :]